# Creation de la base finale pour JS 

Votes et abstention

In [1]:
import pandas as pd
import geopandas as gpd
import os 
import matplotlib.pyplot as plt

In [2]:
dir=os.getcwd()

Densite de population

In [3]:
dens=pd.read_csv(dir+'\\brut\\metropole-densites-de-population-par-commune.csv',sep=';')
dens=dens.rename(columns={'Code INSEE':'insee',"Densité d'habitants (hab/km2)" : 'densitehabkm2'})
dens=dens[['insee','densitehabkm2']]

FileNotFoundError: [Errno 2] File /home/steve/Documents/ENSAE/Data_StoryTelling/dataviz\brut\metropole-densites-de-population-par-commune.csv does not exist: '/home/steve/Documents/ENSAE/Data_StoryTelling/dataviz\\brut\\metropole-densites-de-population-par-commune.csv'

In [ ]:
dens.insee=dens.insee.astype(int)
dens.dtypes

In [ ]:
A=dens.sort_values('insee').sort_values('insee')

In [ ]:
list(A.insee)

Niveau de vie et taux de pauvreté

In [ ]:
com=pd.read_csv(dir+'\\brut\\base-comparateur-de-territoires.csv',sep=';')
com=com.rename(columns={'CODGEO':'insee','MED16':'medianendv','TP6016':'tp'})
com=com[['insee','tp','medianendv']]

In [ ]:
com.insee=com.insee.astype(int)
com.dtypes

In [ ]:
com.head() # tp on a pas pour tout 

In [ ]:
len(com.insee.unique())

Pollution

In [ ]:
pol=pd.read_csv(dir+'\\brut\\indices_QA_commune_IDF_2017.csv',sep=',')

pol=pol.groupby('ninsee',as_index=False).mean()

pol=pol.rename(columns={'ninsee':'insee'})

In [ ]:
pol.insee=pol.insee.astype(int)

In [ ]:
pol.dtypes

Diplomes 

In [ ]:
dipl = pd.read_csv(dir+'\\brut\\base-cc-diplomes-formation-2016.csv',sep=';')

In [ ]:
dipl=dipl[['CODGEO','P16_NSCOL15P','P16_NSCOL15P_SUP']]

dipl=dipl.rename(columns={'P16_NSCOL15P':'nb_nscol','P16_NSCOL15P_SUP':'dipl_sup','CODGEO':'insee'})

dipl=dipl.assign(dipl_su_tx=dipl.dipl_sup/dipl.nb_nscol)

dipl=dipl[['insee','dipl_su_tx']]


In [ ]:
dipl.dtypes

In [ ]:
import numpy as np

In [ ]:
dipl=dipl[pd.to_numeric(dipl['insee'], errors='coerce').notnull()]

In [ ]:
dipl.insee=dipl.insee.astype(int)

Prix au mètre carré 

In [ ]:
prix=pd.read_csv(dir+'\\brut\\prixm2-communes-2017.csv',sep=',')

prix=prix[['INSEE_COM','Prixm2']]

prix=prix.rename(columns={'INSEE_COM':'insee','Prixm2':'prixm2'})

In [ ]:
prix.dtypes

In [ ]:
prix=prix[pd.to_numeric(prix['insee'], errors='coerce').notnull()]
prix.insee=prix.insee.astype(int)

In [ ]:
prix.dtypes

Impôt sur la fortune

In [ ]:
isf=pd.read_excel(dir+'\\brut\\ificom2018.xlsx',headers=True,sheet_name='définitif impot')

isf=isf[['Code commune (INSEE)','nombre de redevables','patrimoine moyen en €','impôt moyen en €']]

isf.columns=['insee','nb_redevables','patrimoineeuro','impotmoyeneuro']

In [ ]:
isf.dtypes

In [ ]:
isf['insee']= isf['insee'].transform(lambda x : x.replace(' ','')) 

In [ ]:
isf=isf[pd.to_numeric(isf['insee'], errors='coerce').notnull()]
isf.insee=isf.insee.astype(int)

In [ ]:
isf.dtypes

In [ ]:
#Base Sarah 
eco=pd.read_csv(dir+'\\brut\\data_chom_prop.csv')
eco.head()

In [ ]:
eco=eco.drop(columns=['Unnamed: 0'])
eco=eco.rename(columns={'CODGEO':'insee'})

In [ ]:
eco.dtypes

Votes

In [ ]:
#vote =pd.read_csv(dir+'\\brut\\data_final_vote.csv')
#vote=vote[['CODGEO','Tour1Candidat1']]

In [ ]:
#vote=vote.rename(columns={'CODGEO':'insee'})

In [ ]:
# Vote à la mano 
vote = pd.read_excel(dir+'\\brut\\vote_mano.xlsx')

Merge 

In [ ]:
bases=[eco,dens,dipl,pol,vote,com,isf,prix]

In [ ]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')
variables= pd.DataFrame(pop.insee)
variables.insee=variables.insee.astype(int)

In [ ]:
pop.plot()

In [ ]:
for data in bases:
    variables=variables.merge(data,how='left',on='insee')

In [ ]:
variables.head()

In [ ]:
# SOucis avec les élections j'ai pas les correspondances !! 

In [ ]:
variables.drop_duplicates( inplace = True) 

In [ ]:
#variables.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
variables = variables.round(2)

In [ ]:
variables.to_csv('variables.csv')

Creation du CSV our la population slider

In [ ]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')

In [ ]:
pop=pop[['insee','year','population','pct_ch','steps']]

In [ ]:
pop.to_csv('population_csv.csv')

In [ ]:
pop[pop.insee=='78561']

Professionnels de Santé

In [ ]:
med=pd.read_csv(dir+'\\brut\\annuaire-et-localisation-des-professionnels-de-sante.csv',sep=';')

med=med[["Nature de l'exercice","Convention du professionnel","Coordonnées",'Adresse','Code INSEE']]

med.columns=['nature_exercice','convention','coordonnees','adresse','insee']

In [ ]:
# Ici interessant de mettre des marqueurs ? Je laisse comme ça => sinon compter 
med.groupby('insee').count()

In [ ]:
med.dtypes

In [ ]:
med.insee=med.insee.astype(int)

# Traffic (lié aux stations et non aux communes)

Cartes pour visualiser le résultat 

In [ ]:
import geopandas as gpd 
pop=gpd.read_file(r'C:\Users\morga\Documents\GitHub\MLPython\dataviz\Graphique_bokeh\pop_comgeo.geojson')
pop.insee=pop.insee.astype(int)
data_map=variables.merge(pop,on='insee')

In [ ]:
data_map.columns

In [ ]:
import mapclassify
import geoplot
import matplotlib.pyplot as plt 
data_map=data_map.set_geometry('geometry')

In [ ]:
scheme = mapclassify.Quantiles(data_map.TauxChomage, k=10)
geoplot.choropleth(
    data_map, hue=data_map.TauxChomage, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('Taux de chômage')

In [ ]:
scheme = mapclassify.Quantiles(data_map.TauxProprietaires, k=10)
geoplot.choropleth(
    data_map, hue=data_map.TauxProprietaires, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('Taux de propriétaires')

In [ ]:
scheme = mapclassify.Quantiles(data_map.pm10, k=10)
geoplot.choropleth(
    data_map, hue=data_map.pm10, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('Taux pollution pm10')

In [ ]:
scheme = mapclassify.Quantiles(data_map.dipl_su_tx, k=10)
geoplot.choropleth(
    data_map, hue=data_map.dipl_su_tx, scheme=scheme,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('dipl_su_tx')

In [ ]:

geoplot.choropleth(
    data_map, hue=data_map.Tour1Candidat1,
    cmap='OrRd', figsize=(10, 5),legend=True
)
plt.title('vote_politique')